In [ ]:
import re
from playwright.sync_api import sync_playwright

URL = "https://iphonekaitori.tokyo/series/iphone/market-price"
TARGET_NAME = "iPhone 17 Pro 256GB シルバー"

def extract_target_row():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        page.goto(URL, wait_until="domcontentloaded")
        page.wait_for_timeout(1000)

        # 1) 找到“iPhone カラー別・ランク別買取価格表”这一段的表格
        #    用 heading 文本定位，再取其后续的第一个 table（更稳一些）
        heading = page.locator("text=iPhone カラー別・ランク別買取価格表").first
        table = heading.locator("xpath=following::table[1]")

        # 2) 在表格中找到包含目标机型名的那一行 tr
        row = table.locator("tr", has=page.locator(f"text={TARGET_NAME}")).first
        row.wait_for(state="visible", timeout=10000)

        # 3) 该行结构（按 td 顺序）大致是：
        #    td[0]=シリーズ, td[1]=キャリア, td[2]=機種名(含型番/JAN), td[3]=未開封, td[4]=未使用, td[5]=ランクA ...
        tds = row.locator("td")
        device_text = tds.nth(2).inner_text().strip()
        price_mikaifu = tds.nth(3).inner_text().strip()  # “未開封”栏

        # 4) 从 device_text 里用正则解析 型番 / JAN
        #    例：
        #    iPhone 17 Pro 256GB シルバー（）
        #
        #    型番：MG854J/A
        #
        #    JANコード：4549995648294
        m_type = re.search(r"型番：\s*([A-Z0-9/]+)", device_text)
        m_jan  = re.search(r"JANコード：\s*([0-9]+)", device_text)

        result = {
            "iphone": TARGET_NAME,
            "type": m_type.group(1) if m_type else None,
            "jan": m_jan.group(1) if m_jan else None,
            "price": price_mikaifu,
        }

        browser.close()
        return result

data = extract_target_row()
data


In [4]:
import re
import pandas as pd
from playwright.async_api import async_playwright

URL = "https://iphonekaitori.tokyo/series/iphone/market-price"

def parse_device_cell(text: str):
    """
    从“機種名”单元格的多行文本中解析：
    - iphone: 机型名（第一行）
    - type: 型番
    - jan: JANコード
    """
    t = (text or "").strip()
    first_line = t.splitlines()[0].strip() if t else ""
    iphone = re.sub(r"\s*（.*?）\s*", "", first_line).strip()

    m_type = re.search(r"型番：\s*([A-Z0-9/]+)", t)
    m_jan  = re.search(r"JANコード：\s*([0-9]+)", t)

    return iphone, (m_type.group(1) if m_type else None), (m_jan.group(1) if m_jan else None)

def yen_to_int(s):
    if s is None:
        return None
    m = re.search(r"([0-9,]+)\s*円", str(s))
    return int(m.group(1).replace(",", "")) if m else None

async def scrape_rank_table_to_df(headless=True) -> pd.DataFrame:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=headless)
        page = await browser.new_page()
        await page.goto(URL, wait_until="domcontentloaded")
        await page.wait_for_timeout(1000)

        # “カラー別・ランク別買取価格表”の直後の table を取る
        heading = page.locator("text=iPhone カラー別・ランク別買取価格表").first
        table = heading.locator("xpath=following::table[1]")
        await table.wait_for(state="visible", timeout=15000)

        rows = table.locator("tr")
        n = await rows.count()

        records = []
        for i in range(n):
            tr = rows.nth(i)
            tds = tr.locator("td")
            td_count = await tds.count()
            if td_count < 4:
                continue

            # td[0]=シリーズ, td[1]=キャリア, td[2]=機種名(型番/JAN含む)
            series = (await tds.nth(0).inner_text()).strip()
            career = (await tds.nth(1).inner_text()).strip()
            device_text = (await tds.nth(2).inner_text()).strip()

            iphone, type_code, jan = parse_device_cell(device_text)

            async def safe_td(idx):
                if td_count > idx:
                    return (await tds.nth(idx).inner_text()).strip()
                return None

            rec = {
                "シリーズ": series,
                "キャリア": career,
                "iphone": iphone,
                "type": type_code,
                "jan": jan,
                "未開封": await safe_td(3),
                "未使用": await safe_td(4),
                "ランクA": await safe_td(5),
                "ランクB": await safe_td(6),
                "ランクC": await safe_td(7),
            }
            records.append(rec)

        await browser.close()

    df = pd.DataFrame.from_records(records)

    # 价格数值列（円）
    for col in ["未開封", "未使用", "ランクA", "ランクB", "ランクC"]:
        df[col + "_int"] = df[col].apply(yen_to_int)

    return df

# ✅ 在 Jupyter 里直接 await
df = await scrape_rank_table_to_df(headless=True)
# target = df[df["iphone"].str.contains("iPhone 17 Pro 256GB シルバー", na=False)]
df[["iphone","type","jan","未開封","未開封_int"]]


,iphone,type,jan,未開封,未開封_int
0,iPhone 17 Pro 256GB シルバー,MG854J/A,4549995648294,"184,000円",184000
1,iPhone 17 Pro 256GB コズミックオレンジ,MG864J/A,4549995648300,"184,000円",184000
2,iPhone 17 Pro 256GB ディープブルー,MG874J/A,4549995648317,"184,000円",184000
3,iPhone 17 Pro 512GB シルバー,MG894J/A,4549995648324,"206,000円",206000
4,iPhone 17 Pro 512GB コズミックオレンジ,MG8A4J/A,4549995648331,"206,000円",206000
...,...,...,...,...,...
102,iPhone 16e 128GB ホワイト,MD1R4J/A,4549995559002,"88,000円",88000
103,iPhone 16e 256GB ブラック,MD1T4J/A,4549995559019,"95,000円",95000
104,iPhone 16e 256GB ホワイト,MD1W4J/A,4549995559026,"95,000円",95000
105,iPhone 16e 512GB ブラック,MD1X4J/A,4549995559033,"101,000円",101000


In [5]:
GOODS_JSON = {
	"code": 1,
	"msg": "获取成功",
	"time": "1770008236",
	"data": {
		"total": 12,
		"per_page": 20,
		"current_page": 1,
		"last_page": 1,
		"data": [
			{
				"goods_id": 36,
				"title": "iPhone Air 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "スペースブラック",
				"price": 229800
			},
			{
				"goods_id": 36,
				"title": "iPhone Air 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "クラウドホワイト",
				"price": 229800
			},
			{
				"goods_id": 36,
				"title": "iPhone Air 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ライトゴールド",
				"price": 229800
			},
			{
				"goods_id": 36,
				"title": "iPhone Air 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 3,
				"spec_name": "スカイブルー",
				"price": 229800
			},
			{
				"goods_id": 35,
				"title": "iPhone Air 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "スペースブラック",
				"price": 194800
			},
			{
				"goods_id": 35,
				"title": "iPhone Air 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "クラウドホワイト",
				"price": 194800
			},
			{
				"goods_id": 35,
				"title": "iPhone Air 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ライトゴールド",
				"price": 194800
			},
			{
				"goods_id": 35,
				"title": "iPhone Air 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 3,
				"spec_name": "スカイブルー",
				"price": 194800
			},
			{
				"goods_id": 34,
				"title": "iPhone Air 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "スペースブラック",
				"price": 159800
			},
			{
				"goods_id": 34,
				"title": "iPhone Air 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "クラウドホワイト",
				"price": 159800
			},
			{
				"goods_id": 34,
				"title": "iPhone Air 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ライトゴールド",
				"price": 159800
			},
			{
				"goods_id": 34,
				"title": "iPhone Air 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c2426e262294d9cec5dbb68769b1631d.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone Air",
				"status": 1,
				"status_text": "上架",
				"spec_index": 3,
				"spec_name": "スカイブルー",
				"price": 159800
			},
			{
				"goods_id": 33,
				"title": "iPhone 17 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "ブラック",
				"price": 164800
			},
			{
				"goods_id": 33,
				"title": "iPhone 17 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "ホワイト",
				"price": 164800
			},
			{
				"goods_id": 33,
				"title": "iPhone 17 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ミストブルー",
				"price": 164800
			},
			{
				"goods_id": 33,
				"title": "iPhone 17 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 3,
				"spec_name": "ラベンダー",
				"price": 164800
			},
			{
				"goods_id": 33,
				"title": "iPhone 17 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 4,
				"spec_name": "セージ",
				"price": 164800
			},
			{
				"goods_id": 32,
				"title": "iPhone 17 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "ブラック",
				"price": 129800
			},
			{
				"goods_id": 32,
				"title": "iPhone 17 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "ホワイト",
				"price": 129800
			},
			{
				"goods_id": 32,
				"title": "iPhone 17 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ミストブルー",
				"price": 129800
			},
			{
				"goods_id": 32,
				"title": "iPhone 17 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 3,
				"spec_name": "ラベンダー",
				"price": 129800
			},
			{
				"goods_id": 32,
				"title": "iPhone 17 256G",
				"image": "http://www.mobile-zone.jp/uploads/20251118/be9b4a814a784dd231dff2d40d5bfa98.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17",
				"status": 1,
				"status_text": "上架",
				"spec_index": 4,
				"spec_name": "セージ",
				"price": 129800
			},
			{
				"goods_id": 31,
				"title": "iPhone 17 Pro 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 214800
			},
			{
				"goods_id": 31,
				"title": "iPhone 17 Pro 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 214800
			},
			{
				"goods_id": 31,
				"title": "iPhone 17 Pro 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 214800
			},
			{
				"goods_id": 30,
				"title": "iPhone 17 Pro 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 179800
			},
			{
				"goods_id": 30,
				"title": "iPhone 17 Pro 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 179800
			},
			{
				"goods_id": 30,
				"title": "iPhone 17 Pro 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 179800
			},
			{
				"goods_id": 29,
				"title": "iPhone 17 Pro Max 2TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 329800
			},
			{
				"goods_id": 29,
				"title": "iPhone 17 Pro Max 2TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 329800
			},
			{
				"goods_id": 29,
				"title": "iPhone 17 Pro Max 2TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 329800
			},
			{
				"goods_id": 28,
				"title": "iPhone 17 Pro Max 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 264800
			},
			{
				"goods_id": 28,
				"title": "iPhone 17 Pro Max 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 264800
			},
			{
				"goods_id": 28,
				"title": "iPhone 17 Pro Max 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 264800
			},
			{
				"goods_id": 27,
				"title": "iPhone 17 Pro Max 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 229800
			},
			{
				"goods_id": 27,
				"title": "iPhone 17 Pro Max 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 229800
			},
			{
				"goods_id": 27,
				"title": "iPhone 17 Pro Max 512GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 229800
			},
			{
				"goods_id": 24,
				"title": "iPhone 17 Pro Max 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 194800
			},
			{
				"goods_id": 24,
				"title": "iPhone 17 Pro Max 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 194800
			},
			{
				"goods_id": 24,
				"title": "iPhone 17 Pro Max 256GB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/0343c434b9679cc33f33935d0c53437c.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro Max",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 194800
			},
			{
				"goods_id": 22,
				"title": "iPhone 17 Pro 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 0,
				"spec_name": "シルバー",
				"price": 209800
			},
			{
				"goods_id": 22,
				"title": "iPhone 17 Pro 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 1,
				"spec_name": "コズミックオレンジ",
				"price": 219800
			},
			{
				"goods_id": 22,
				"title": "iPhone 17 Pro 1TB",
				"image": "http://www.mobile-zone.jp/uploads/20251118/c495e0f0370beda83140f33a0bb65238.webp",
				"category_name": "iPhone",
				"category_second_name": "iPhone 17 シリーズ",
				"category_three_name": "iPhone 17 Pro",
				"status": 1,
				"status_text": "上架",
				"spec_index": 2,
				"spec_name": "ディープブルー",
				"price": 239800
			}
		]
	}
}

In [6]:
import re
import pandas as pd
from typing import Optional, Tuple, Dict

def _norm_space(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").replace("\u3000", " ")).strip()

def _norm_title(s: str) -> str:
    """
    统一 title 格式以便匹配：
    - 去多余空白
    - 统一 256G / 256GB / 256g => 256gb（同理 1TB）
    - 统一大小写
    """
    s = _norm_space(s).lower()
    # 256g -> 256gb
    s = re.sub(r"(\d+)\s*g\b", r"\1gb", s)
    # 256 gb -> 256gb
    s = re.sub(r"(\d+)\s*gb\b", r"\1gb", s)
    # 1 tb -> 1tb
    s = re.sub(r"(\d+)\s*tb\b", r"\1tb", s)
    return s

def _norm_color(s: str) -> str:
    return _norm_space(s)

def parse_iphone_to_title_and_color(iphone: str) -> Tuple[Optional[str], Optional[str]]:
    """
    从 df['iphone']（例如：'iPhone 17 Pro 256GB シルバー'）解析：
    - title_candidate: 'iPhone 17 Pro 256GB'
    - color_candidate: 'シルバー'
    规则：默认把最后一个 token 当作颜色（spec_name），其余部分当作 title。
    """
    s = _norm_space(iphone)
    if not s:
        return None, None
    parts = s.split(" ")
    if len(parts) < 2:
        return s, None
    color = parts[-1]
    title = " ".join(parts[:-1])
    return title, color

def build_goods_index(goods_json: dict) -> Dict[Tuple[str, str], Tuple[int, int]]:
    """
    建立索引：(norm_title, norm_spec_name) -> (goods_id, spec_index)
    """
    items = goods_json.get("data", {}).get("data", [])
    idx = {}
    for it in items:
        title = _norm_title(it.get("title", ""))
        spec = _norm_color(it.get("spec_name", ""))
        goods_id = it.get("goods_id", None)
        spec_index = it.get("spec_index", None)
        if title and spec and goods_id is not None and spec_index is not None:
            idx[(title, spec)] = (int(goods_id), int(spec_index))
    return idx

def add_goods_mapping(df: pd.DataFrame, goods_json: dict) -> pd.DataFrame:
    """
    在 df 中新增：goods_id, spec_index
    依据：df['iphone'] -> (title_candidate, color_candidate) -> 在 goods_json 中匹配 title+spec_name
    """
    goods_index = build_goods_index(goods_json)

    def mapper(iphone: str):
        title_raw, color_raw = parse_iphone_to_title_and_color(iphone)
        if not title_raw or not color_raw:
            return (pd.NA, pd.NA)

        key = (_norm_title(title_raw), _norm_color(color_raw))

        # 直接命中
        if key in goods_index:
            gid, sidx = goods_index[key]
            return (gid, sidx)

        # 兜底：有时 df 的 title 会是 256GB，但 JSON 里可能写 256G（或反之）
        # 我们做一次“弱等价”尝试：把 gb <-> g 再试一次
        title_norm = key[0]
        alt_title_norm = (
            re.sub(r"(\d+)gb\b", r"\1g", title_norm)
            if "gb" in title_norm
            else re.sub(r"(\d+)g\b", r"\1gb", title_norm)
        )
        alt_key = (alt_title_norm, key[1])
        if alt_key in goods_index:
            gid, sidx = goods_index[alt_key]
            return (gid, sidx)

        return (pd.NA, pd.NA)

    out = df.copy()
    mapped = out["iphone"].apply(mapper)
    out["goods_id"] = mapped.apply(lambda x: x[0])
    out["spec_index"] = mapped.apply(lambda x: x[1])
    return out


In [9]:
df2 = add_goods_mapping(df, GOODS_JSON)
df2[["iphone","type","jan","未開封","未開封_int","goods_id","spec_index"]].head()

,iphone,type,jan,未開封,未開封_int,goods_id,spec_index
0,iPhone 17 Pro 256GB シルバー,MG854J/A,4549995648294,"184,000円",184000,30,0
1,iPhone 17 Pro 256GB コズミックオレンジ,MG864J/A,4549995648300,"184,000円",184000,30,1
2,iPhone 17 Pro 256GB ディープブルー,MG874J/A,4549995648317,"184,000円",184000,30,2
3,iPhone 17 Pro 512GB シルバー,MG894J/A,4549995648324,"206,000円",206000,31,0
4,iPhone 17 Pro 512GB コズミックオレンジ,MG8A4J/A,4549995648331,"206,000円",206000,31,1


In [12]:
import math
import requests
import pandas as pd

API_URL = "http://www.mobile-zone.jp/api/goodsprice/update"
TOKEN = "008c43ec-7b08-4af0-86c4-b4495e15cee0"

def build_prices_payload(
    df2: pd.DataFrame,
    price_col: str = "未開封_int",
    drop_zero_or_null: bool = True,
) -> dict:
    """
    从 df2 构造 API payload:
      {
        "prices": [{"goods_id": int, "spec_index": int, "price": float}, ...]
      }
    """
    required_cols = {"goods_id", "spec_index", price_col}
    missing = required_cols - set(df2.columns)
    if missing:
        raise ValueError(f"df2 缺少必要列: {missing}")

    d = df2.copy()

    # 只保留可映射行
    d = d[d["goods_id"].notna() & d["spec_index"].notna() & d[price_col].notna()]

    # 转为数值
    d["goods_id"] = d["goods_id"].astype(int)
    d["spec_index"] = d["spec_index"].astype(int)
    d[price_col] = pd.to_numeric(d[price_col], errors="coerce")

    if drop_zero_or_null:
        d = d[d[price_col].notna() & (d[price_col] > 0)]

    prices = [
        {
            "goods_id": int(row["goods_id"]),
            "spec_index": int(row["spec_index"]),
            "price": float(row[price_col]),  # API 示例是 209800.00
        }
        for _, row in d.iterrows()
    ]

    return {"prices": prices}

def post_update_prices(payload: dict, token: str, api_url: str = API_URL, timeout: int = 30):
    headers = {
        "token": token,
        "Content-Type": "application/json",
    }
    resp = requests.post(api_url, headers=headers, json=payload, timeout=timeout)
    # 非 2xx 直接抛错，方便你在 notebook 里看到原因
    resp.raise_for_status()
    return resp.json()

# 1) 组装 payload（默认用 未開封_int）
payload = build_prices_payload(df2, price_col="未開封_int", drop_zero_or_null=True)

# 看看本次要更新多少条
len(payload["prices"]), payload["prices"]


(43,
 [{'goods_id': 30, 'spec_index': 0, 'price': 184000.0},
  {'goods_id': 30, 'spec_index': 1, 'price': 184000.0},
  {'goods_id': 30, 'spec_index': 2, 'price': 184000.0},
  {'goods_id': 31, 'spec_index': 0, 'price': 206000.0},
  {'goods_id': 31, 'spec_index': 1, 'price': 206000.0},
  {'goods_id': 31, 'spec_index': 2, 'price': 206000.0},
  {'goods_id': 22, 'spec_index': 0, 'price': 236000.0},
  {'goods_id': 22, 'spec_index': 1, 'price': 236000.0},
  {'goods_id': 22, 'spec_index': 2, 'price': 236000.0},
  {'goods_id': 24, 'spec_index': 0, 'price': 204000.0},
  {'goods_id': 24, 'spec_index': 1, 'price': 203000.0},
  {'goods_id': 24, 'spec_index': 2, 'price': 204000.0},
  {'goods_id': 27, 'spec_index': 0, 'price': 226000.0},
  {'goods_id': 27, 'spec_index': 1, 'price': 225000.0},
  {'goods_id': 27, 'spec_index': 2, 'price': 226000.0},
  {'goods_id': 28, 'spec_index': 0, 'price': 257000.0},
  {'goods_id': 28, 'spec_index': 1, 'price': 257000.0},
  {'goods_id': 28, 'spec_index': 2, 'price'

In [11]:
result = post_update_prices(payload, token=TOKEN)
result

{'code': 1,
 'msg': '成功更新了12个商品',
 'time': '1770013006',
 'data': {'updated_count': 12,
  'updated_goods': [30, 31, 22, 24, 27, 28, 29, 34, 35, 36, 32, 33]}}